#### Trains our transformer model on the tokenized data

1. Loads our token sequences and vocabulary 
2. Build Transformer model (probably going to use Pytorch but we will look into what)
3. Train on tokens [:-1] -> [tokens [1:]]
4. Save checkpoints and training logs


P.S This is subject to lots of change on how we prepare the model I think we will probably be training on multiple datasets and then freezing at different points
i.e train first on the accompaniment dataset so it understands general structure and how accompaniment works, then train on dataset with melodies as well so it gets how 
to play and be reactive based on the melody (we also need to add in a lot of noise and personal data for this part since we will not be perfect soloist like in the jazz
songs and MIDI from a Database)

Load the token sequences

In [2]:
import json
from pathlib import Path

In [3]:
project_root = Path.cwd().parent
remi_segments_path = project_root / "remi_segments.jsonl"
vocab_path = project_root / "vocab.json"

# Load vocab
with open(vocab_path, "r") as f:
    vocab = json.load(f)
vocab_size = len(vocab)

# Load token sequences
token_sequences = []
with open(remi_segments_path, "r") as f:
    for line in f:
        record = json.loads(line)
        token_sequences.append(record["tokens"])

Encode the tokens as integers

In [4]:
def encode(tokens):
    return [vocab.get(t, vocab["<UNK>"]) for t in tokens]

encoded_sequences = [encode(seq) for seq in token_sequences]

Training Data

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class TokenDataset(Dataset):
    def __init__(self, sequences, seq_len=512):
        self.samples = []
        for seq in sequences:
            # Chop long sequences into chunks
            for i in range(0, len(seq) - seq_len):
                self.samples.append((
                    seq[i:i+seq_len],
                    seq[i+1:i+seq_len+1]
                ))
    def __len__(self):
        return len(self.samples)
    def __getitem__(self, idx):
        x, y = self.samples[idx]
        return torch.tensor(x), torch.tensor(y)

seq_len = 128  # or 512, depending on your GPU
dataset = TokenDataset(encoded_sequences, seq_len=seq_len)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)